# CJ더마켓 카테고리&상품명 크롤링

## 0. 기본 설정

In [1]:
# 크롤링에 필요한 함수 불러오기
# selenium은 제어하고자 하는 웹 브라우저의 driver를 설치해야 함 (일반적으로 Chrome 사용)
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By

import time

In [2]:
DRIVER_PATH = "C:/Users/USER/Chromedriver/chromedriver.exe"

chrome_options = Options()
chrome_options.add_argument( '--headless' )
chrome_options.add_argument( '--log-level=3' )
chrome_options.add_argument( '--disable-logging' )
chrome_options.add_argument( '--no-sandbox' )
chrome_options.add_argument( '--disable-gpu' )

# selenium webdriver 위치 지정
driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=chrome_options)

C:\Users\USER\AppData\Local\Temp\ipykernel_20332\946588490.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=chrome_options)


## 1. 크롤링 함수 만들기

In [3]:
def crowl_category(driver, url, cate_list):
    # selenium driver로 url 접속
    driver.get(url)
    time.sleep(2)

    # 카테고리 페이지 개수 가져오기
    pages = len(driver.find_element(By.CLASS_NAME, 'paging').find_elements(By.CSS_SELECTOR, 'li'))
    if pages == 0:
        pages = 1

    # 페이지 돌면서 데이터 가져오기
    for j in range(1, pages+1):
        print(f'    page = {j}')
        driver.get(url + str(j))
        time.sleep(2)
        
        elements = driver.find_element(By.CLASS_NAME, 'product-list').find_elements_by_css_selector('li.product-item')
        print('     상품 개수: {}'.format(len(elements)))
        # 해당 카테고리에 속하는 상품들의 이름을 리스트에 담기
        for el in elements:
            el_title = el.find_element_by_css_selector('div.product-list-cont > a > p.product-title')
            cate_list.append(el_title.text)

## 2. 데이터 크롤링하기

In [4]:
category = {18: '밥/죽/면', 19: '국/김치/김/반찬/두부', 20: '만두/피자/치킨', 21: '핫도그/떡볶이/간식', 22: '돈까스/함박/구이', 23: '스팸/닭가슴살/소시지', 4: '양념/소스/가루/오일', 6: '건강식품', 7: '신선식품', 5: '음료/생수/시럽', 9: '대용량 식자재', 10: '밀키트'}

# 카테고리 개수만큼 리스트에 공간 만들기
cate_list = []
for i in range(len(category.keys())):
    ls = []
    cate_list.append(ls)
    
URL = "https://www.cjthemarket.com/pc/ctg/ctgList?ctgrId=00"

for num, list in zip(category.keys(), cate_list):
    url = URL + f'{num:02d}' + '&page='
    print(f'카테고리 - {category[num]}')
    crowl_category(driver, url, list)

카테고리 - 밥/죽/면
    page = 1
     상품 개수: 80
    page = 2
     상품 개수: 80
    page = 3
     상품 개수: 80
    page = 4
     상품 개수: 0
    page = 5
     상품 개수: 16
카테고리 - 국/김치/김/반찬/두부
    page = 1
     상품 개수: 0
    page = 2
     상품 개수: 80
    page = 3
     상품 개수: 0
    page = 4
     상품 개수: 47
카테고리 - 만두/피자/치킨
    page = 1
     상품 개수: 80
    page = 2
     상품 개수: 80
카테고리 - 핫도그/떡볶이/간식
    page = 1
     상품 개수: 80
    page = 2
     상품 개수: 63
카테고리 - 돈까스/함박/구이
    page = 1
     상품 개수: 35
카테고리 - 스팸/닭가슴살/소시지
    page = 1
     상품 개수: 80
    page = 2
     상품 개수: 11
카테고리 - 양념/소스/가루/오일
    page = 1
     상품 개수: 80
    page = 2
     상품 개수: 80
    page = 3
     상품 개수: 80
    page = 4
     상품 개수: 80
    page = 5
     상품 개수: 12
카테고리 - 건강식품
    page = 1
     상품 개수: 80
    page = 2
     상품 개수: 11
카테고리 - 신선식품
    page = 1
     상품 개수: 80
    page = 2
     상품 개수: 80
    page = 3
     상품 개수: 2
카테고리 - 음료/생수/시럽
    page = 1
     상품 개수: 80
    page = 2
     상품 개수: 31
카테고리 - 대용량 식자재
    page = 1
     상품 개수: 80
    page = 2
  

In [5]:
import pandas as pd

# 카테고리&상품명 정보가 담긴 df 생성하기
ls = []
for cate_name, cate_ls in zip(category.values(), cate_list):
    cate = pd.DataFrame(cate_ls, columns=['product_name'])
    cate['category'] = cate_name
    ls.append(cate)

pd.concat(ls, axis=0)

,product_name,category
0,비비고 평양냉면 541g,밥/죽/면
1,햇반소프트밀 양송이 크림수프*3EA,밥/죽/면
2,햇반소프트밀 베이컨감자 크림수프*3EA,밥/죽/면
3,햇반소프트밀 양송이 크림수프 180g,밥/죽/면
4,햇반소프트밀 베이컨 감자 크림수프 180g,밥/죽/면
...,...,...
25,[지정일 택배배송][쿡킷] 청경채 소고기볶음,밀키트
26,[지정일 택배배송][쿡킷] 더건강한 닭가슴살 토마토리조또,밀키트
27,[지정일 택배배송][쿡킷] 칼칼한 순두부 스팸김치찜,밀키트
28,[택배배송] 얼큰 버섯 비비고 만두 전골_밀키트,밀키트


In [6]:
# csv 파일로 저장하기
pd.concat(ls, axis=0).to_csv('./category_info.csv', index=False, encoding='utf-8-sig')